## Scrape all Oscar acceptance speeches

In [8]:
# Import packages

from selenium.webdriver.common.keys import Keys
# auxiliary functions modified by Luis.
import sys
import calendar
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
import re
from selenium.webdriver.support.ui import Select
import random
import time
from selenium import webdriver
import os
import numpy as np

import requests
from bs4 import BeautifulSoup


from datetime import datetime
from tqdm import tqdm

from pandas.core.common import flatten
import pandas as pd

In [9]:
# set up webdriver path
DRIVER_BIN = '/Users/pazma/Documents/BSE/text_mining/class_4/geckodriver'

In [10]:
def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element_by_xpath(xpath).click()
        elif type == "id":
            browser.find_element_by_id(xpath).click()
        elif type == "css":
            browser.find_element_by_css_selector(xpath).click()
        elif type == "class":
            browser.find_element_by_class_name(xpath).click()
        elif type == "link":
            browser.find_element_by_link_text(xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException):
        return False
    return True


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            #return NoSuchElementException
            ck = True
            # browser.quit()

In [23]:
def scrapeOscars(year):
        year = str(year)
        # Loads up Selenium's WebDriver implementation for Google Chrome
        driver = webdriver.Firefox(executable_path = DRIVER_BIN)

        try:
            # Directs the browser to the Oscars DB Site
            url = "http://aaspeechesdb.oscars.org/"
            driver.get(url)

            # Selects the Input Box corresponding to the Awards Year
            year_xpath = '//input[@id="QI0"]'
            r = driver.find_element_by_xpath(year_xpath)
            r.send_keys(year)

            # Finds the Search button, and clicks it
            #driver.find_element_by_xpath("//input[@id='body_SearchButton']").click()
            check_and_click(driver,"//input[@id='body_SearchButton']" , "xpath")

            # Gets all of the links to each speech made at the Oscars that year
            speechLinkElements = driver.find_elements_by_xpath("//div[@id='main']/div/p")
            print("Number of speeches found: ", len(speechLinkElements))

            # Scrapes the speech data and metadata for each link
            speechResult = []
            for i in range(0, len(speechLinkElements)):
                # Finds the link to get to speech, and clicks on it
                speechLinkElements[i].find_element_by_tag_name("a").click()
                # Finds the div containing all relevant text, and scrapes it
                yearData = driver.find_element_by_xpath("//div[@class='fullModule2 fullContainer']").text
                # Appends the data to the speechResult list
                speechResult.append(yearData)
                # Makes the driver go back, and repeats for a different link
                driver.back()
                # Rescrapes all the links (necessary to keep elements not to be 'stale'/'expired')
                speechLinkElements = driver.find_elements_by_xpath("//div[@id='main']/div/p")

        # except Exception as e:
        #     print("Exception hit when scraping {}".format(year), e)
        #     pass
        #     # driver.close()
        #     # sys.exit(1)

        driver.close()
        return speechResult


In [24]:
start_time = datetime.now()

results=[]
for i in tqdm(range(1939, 2022)): 
    results.append(scrapeOscars(i))

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

with open('speeches.txt', 'w') as f:
    for item in results:
        f.write("%s\n" % item)

  0%|                                                    | 0/83 [00:00<?, ?it/s]/var/folders/_d/v9yy531x3dg4b8kfprprmn280000gn/T/ipykernel_14139/1391933109.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path = DRIVER_BIN)
/var/folders/_d/v9yy531x3dg4b8kfprprmn280000gn/T/ipykernel_14139/1391933109.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  r = driver.find_element_by_xpath(year_xpath)
/var/folders/_d/v9yy531x3dg4b8kfprprmn280000gn/T/ipykernel_14139/968695101.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_xpath(xpath).click()
/var/folders/_d/v9yy531x3dg4b8kfprprmn280000gn/T/ipykernel_14139/1391933109.py:22: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  speechLinkElements = driver.find_elements_by_xpath("//

Duration: 1:01:49.589029


****************

### EXTRA
#### Search by different cetegories and scrape the data according to that: 
    - year
    - actor/actress
    - film title
    - award category  

In [43]:
text=input("What do you want to search for? \n Choose one from the following and type their number:\n1.year\n2.actor/actress\n3.film title\n4.award category\n")

if text=='1':
    searchWord=input("Which year?\n")
elif text=='2':
    searchWord=input("Whose speeches?\n")
elif text=='3':
    searchWord=input("Which film?\n")
elif text=='4':
        searchWord=input("Which award category?\n")    

start_time = datetime.now()


driver = webdriver.Firefox(executable_path = DRIVER_BIN)
driver.get('http://aaspeechesdb.oscars.org/')

if text=='1': 

# Selects the Input Box corresponding to the input text
    year_xpath = '//input[@id="QI0"]'
    r = driver.find_element_by_xpath(year_xpath)
    r.send_keys(searchWord)  # types * to get all the speeches from the very beginning

elif text=='2':
    actor_xpath = '//input[@id="QI1"]'
    r = driver.find_element_by_xpath(actor_xpath)
    r.send_keys(searchWord)

elif text=='3':
    title_xpath = '//input[@id="QI2"]'
    r = driver.find_element_by_xpath(title_xpath)
    r.send_keys(searchWord)

elif text=='4':
    award_xpath = '//input[@id="QI3"]'
    r = driver.find_element_by_xpath(award_xpath)
    r.send_keys(searchWord)


# Finds the Search button, and clicks it
driver.find_element_by_xpath("//input[@id='body_SearchButton']").click()

# Gets all of the links to each speech made at the Oscars 
speechLinkElements = driver.find_elements_by_xpath("//div[@id='main']/div/p")
print("Number of speeches found: ", len(speechLinkElements))

# Scrapes the speech data and metadata for each link
speechResult = []
for i in range(len(speechLinkElements)):
    # Finds the link to get to speech, and clicks on it
    speechLinkElements[i].find_element_by_tag_name("a").click()
    time.sleep(5)
    # Finds the div containing all relevant text, and scrapes it
    speechData = driver.find_element_by_xpath("//div[@class='fullModule2 fullContainer']").text
    # Appends the data to the speechResult list
    speechResult.append(speechData)
    # Makes the driver go back, and repeats for a different link
    # one step back in browser history
    time.sleep(5)
    driver.back()
    # Rescrapes all the links (necessary to keep elements not 'stale')
    speechLinkElements = driver.find_elements_by_xpath("//div[@id='main']/div/p")


driver.close()

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

# export file into txt
with open('speeches_'+searchWord+'.txt', 'w') as f:
    for item in speechResult:
        f.write("%s\n" % item)


What do you want to search for? 
 Choose one from the following and type their number:
1.year
2.actor/actress
3.film title
4.award category
1
Which year?
1939


/var/folders/hc/qzy2rzvj5cg38v_607m73p640000gn/T/ipykernel_7419/3261347139.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path = DRIVER_BIN)
/var/folders/hc/qzy2rzvj5cg38v_607m73p640000gn/T/ipykernel_7419/3261347139.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  r = driver.find_element_by_xpath(year_xpath)
/var/folders/hc/qzy2rzvj5cg38v_607m73p640000gn/T/ipykernel_7419/3261347139.py:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//input[@id='body_SearchButton']").click()
/var/folders/hc/qzy2rzvj5cg38v_607m73p640000gn/T/ipykernel_7419/3261347139.py:45: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  speechLinkElements = driver.find_elements_by_xpath("//div[@id='main']/div/p")


Number of speeches found:  2


/var/folders/hc/qzy2rzvj5cg38v_607m73p640000gn/T/ipykernel_7419/3261347139.py:55: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  speechData = driver.find_element_by_xpath("//div[@class='fullModule2 fullContainer']").text
/var/folders/hc/qzy2rzvj5cg38v_607m73p640000gn/T/ipykernel_7419/3261347139.py:63: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  speechLinkElements = driver.find_elements_by_xpath("//div[@id='main']/div/p")


Duration: 0:00:30.475350


In [21]:
print(len(speechResult))
speechResult

2


['Year: 1939 (12th) Academy Awards\nCategory: Actress\nFilm Title: Gone with the Wind\nWinner: Vivien Leigh\nPresenter: Spencer Tracy\nDate & Venue: February 29, 1940; Ambassador Hotel, Cocoanut Grove (banquet)\n\nVIVIEN LEIGH:\nLadies and gentlemen, please forgive me if my words are inadequate in thanking you for your very great kindness. If I were to mention all those who have shown me such wonderful generosity through "Gone with the Wind" I should have to entertain you with an oration as long as "Gone with the Wind" itself. So if I may, I should like to devote my thanks on this occasion to that composite figure of energy, courage, and very great kindness, in whom all points of "Gone with the Wind" meet -- Mr. David Selznick.\n© Academy of Motion Picture Arts and Sciences\n[Note: All winners are present except where noted; NOT all winners may have spoken.]',
 'Watch the video\nYear: 1939 (12th) Academy Awards\nCategory: Actress in a Supporting Role\nFilm Title: Gone with the Wind\nWi

**********************

### Minor parsing attempt (not fully  finished)

In [23]:
import re

with open('/Users/pazma/Downloads/speeches.txt') as f:
    content = [line.rstrip('\n') for line in f]

In [24]:
speeches=[]
for i in range(len(content)):
    speeches.append(content[i].split("\\n"))

r=list(flatten(speeches))

In [25]:
r[:30]

["['Year: 1939 (12th) Academy Awards",
 'Category: Actress',
 'Film Title: Gone with the Wind',
 'Winner: Vivien Leigh',
 'Presenter: Spencer Tracy',
 'Date & Venue: February 29, 1940; Ambassador Hotel, Cocoanut Grove (banquet)',
 '',
 'VIVIEN LEIGH:',
 'Ladies and gentlemen, please forgive me if my words are inadequate in thanking you for your very great kindness. If I were to mention all those who have shown me such wonderful generosity through "Gone with the Wind" I should have to entertain you with an oration as long as "Gone with the Wind" itself. So if I may, I should like to devote my thanks on this occasion to that composite figure of energy, courage, and very great kindness, in whom all points of "Gone with the Wind" meet -- Mr. David Selznick.',
 '© Academy of Motion Picture Arts and Sciences',
 "[Note: All winners are present except where noted; NOT all winners may have spoken.]', 'Watch the video",
 'Year: 1939 (12th) Academy Awards',
 'Category: Actress in a Supporting Rol

In [26]:
speakers=[]
for i in range(len(r)):
    if r[i].isupper()==True:
        speakers.append(r[i])


year=[]
for i in range(len(r)): 
    if re.search(r'(?<=Year: ).*',r[i]) != None:
        year.append(r[i])

date=[]
for i in range(len(r)): 
    if re.search(r'(?<=Date).*',r[i]) != None:
        date.append(r[i])

category=[]
for i in range(len(r)): 
    if re.search(r'(?<=Category).*',r[i]) != None:
        category.append(r[i])

film_title=[]
for i in range(len(r)): 
    if re.search(r'(?<=Film Title).*',r[i]) != None:
        film_title.append(r[i])


winner=[]
for i in range(len(r)): 
    if re.search(r'(?<=Winner).*',r[i]) != None:
        winner.append(r[i])


presenter=[]
for i in range(len(r)): 
    if re.search(r'(?<=Presenter).*',r[i]) != None:
        presenter.append(r[i])


print("# dates and venues: ", len(date))
print("# film titles: ",len(film_title))
print("# categories: ",len(category)) 
print("# speakers: ",len(speakers))
print("# years:", len(year)) 
print("# winners:", len(winner))
print("# presenters: ", len(presenter))



# dates and venues:  1952
# film titles:  1693
# categories:  1951
# speakers:  2706
# years: 1951
# winners: 1990
# presenters:  1964


In [41]:
years = []
for item in year:
    for subitem in item.split():
        if(subitem.isdigit()):
            years.append(subitem)

In [42]:
list(sorted((set(years))))[:10]

['1939',
 '1941',
 '1942',
 '1943',
 '1946',
 '1947',
 '1949',
 '1950',
 '1951',
 '1952']

The lists have different lengths so cannot be put into a dataframe. The reason for that is probably in some cases there were more than one winner/presenter/speaker...etc. 


Also the number of film titles is fewer that the actual number of speeches were given. This needs to be investigated more in depth. 

So we need a slightly different approach. 